In [28]:
# load credentials from environment variables
%load_ext dotenv
%dotenv

from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [43]:


!rm -r *.zarr
request_collector = Observers.request_dumper()

for band in ['B04']:
    config = CubeConfig(
        dataset_name="S2L2A",
        band_names=[band],
        geometry=(14.318395625014208,48.359153946428485,14.506361110382839,48.543395524629055),
        #tile_size=[1024, 12],
        crs='http://www.opengis.net/def/crs/EPSG/0/4326',
        spatial_res=0.000134,
        time_range=['2018-01-01', '2018-01-31'],
        time_period='2W'
    )
    open_cube(config, observer=request_collector).to_zarr(f'{band}.zarr')
    #print(request_collector.stats.duration_mean*request_collector.stats.num_requests)
    %time !xcube sh gen S2L2A -v -o ooe_{band}.zarr -b {band} -g 14.318395625014208,48.359153946428485,14.506361110382839,48.543395524629055 -r 0.000134 -t 2018-01-01,2018-01-31 -p 2W 2>/dev/null
    
!du *.zarr -s

Received chunk (0, 0, 0) for band B04: took 1583.62 ms
Received chunk (1, 0, 0) for band B04: took 2838.19 ms
Received chunk (2, 0, 0) for band B04: took 1813.20 ms
Writing cube to ooe_B04.zarr...
Received chunk (1, 0, 0) for band B04: took 1360.07 ms
Received chunk (0, 0, 0) for band B04: took 1841.93 ms
Received chunk (2, 0, 0) for band B04: took 1400.93 ms
Cube written to ooe_B04.zarr, took 4.17 seconds.
Number of requests: 3
Request duration min: 1360.07 ms
Request duration max: 1841.93 ms
Request duration median: 1400.93 ms
Request duration mean: 1534.31 ms
Request duration std.dev.: 218.16 ms
CPU times: user 98.8 ms, sys: 17.3 ms, total: 116 ms
Wall time: 5.86 s
14648	B04.zarr
14648	ooe_B04.zarr
